In [3]:
import pandas as pd
import matplotlib.pyplot as pyplt
import numpy as np
import seaborn

data = pd.read_csv("data.csv")


7d2b2dbbfb2045f7bf69a01cfacf0e8b


,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,DÁKITI,EL ÚLTIMO TOUR DEL MUNDO,Bad Bunny,2020-11-27,205090,86,0.731,0.4010,0.731,0.573,0.000052,0.1130,-10.059,0.0544,109.928,4
1,LA NOCHE DE ANOCHE,EL ÚLTIMO TOUR DEL MUNDO,Bad Bunny,2020-11-27,203200,91,0.856,0.0303,0.856,0.618,0.000000,0.0866,-4.892,0.2860,81.993,4
2,positions,Positions,Ariana Grande,2020-10-30,172324,97,0.737,0.4680,0.737,0.802,0.000000,0.0931,-4.771,0.0878,144.015,4
3,All I Want for Christmas Is You,Merry Christmas,Mariah Carey,1994-11-01,241106,94,0.336,0.1640,0.336,0.627,0.000000,0.0708,-7.463,0.0384,150.273,4
4,Mood (feat. iann dior),Mood (feat. iann dior),24kGoldn,2020-07-24,140525,98,0.700,0.2210,0.700,0.722,0.000000,0.2720,-3.558,0.0369,90.989,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,De Cora <3,Afrodisíaco,Rauw Alejandro,2020-11-13,190100,82,0.811,0.4600,0.811,0.683,0.000048,0.1280,-2.028,0.2410,176.037,4
196,hot girl bummer,everything means nothing,blackbear,2020-08-21,188343,86,0.782,0.1250,0.782,0.559,0.000000,0.3850,-7.106,0.0767,129.992,1
197,All of Me,Love In The Future (Expanded Edition),John Legend,2013-08-30,269560,86,0.422,0.9220,0.422,0.264,0.000000,0.1320,-7.064,0.0322,119.930,4
198,Star Shopping,Star Shopping,Lil Peep,2019-04-19,142000,86,0.585,0.0967,0.585,0.471,0.018400,0.1150,-9.934,0.0616,93.099,4
